# FRB 190520 (approximate)

In [10]:
# imports
from importlib import reload
import os
from pkg_resources import resource_filename

import numpy as np

import pandas

from astropy.coordinates import SkyCoord
from astropy import units

from frb.galaxies import nebular

from astropath import path
from astropath import localization

# Instantiate

In [3]:
Path = path.PATH()

# FRB

## FRB Coord

In [5]:
frb_coord = SkyCoord('16h02m04.268s -11d17m17.32s', frame='icrs')
frb_coord

<SkyCoord (ICRS): (ra, dec) in deg
    (240.51778333, -11.28814444)>

## Error

In [6]:
eellipse = dict(a=0.125, b=0.125, theta=0.)

In [8]:
Path.init_localization('eellipse', center_coord=frb_coord, eellipse=eellipse)
Path.localiz

{'type': 'eellipse',
 'center_coord': <SkyCoord (ICRS): (ra, dec) in deg
     (240.51778333, -11.28814444)>,
 'eellipse': {'a': 0.125, 'b': 0.125, 'theta': 0.0}}

# Galactic Extinction

In [12]:
nebular.get_ebv(frb_coord)

{'refPixelValue': 0.2866,
 'meanValue': 0.2903,
 'std': 0.005,
 'maxValue': 0.3026,
 'minValue': 0.2808}

# Candidates (kludgy)

## From the paper

In [14]:
gal1_coord = SkyCoord('J160204.31-111718.5',  unit=(units.hourangle, units.deg))
gal1_r = 21.  #

In [23]:
gal2_coord = gal1_coord.directional_offset_by(135.*units.deg, 1*units.arcsec)
gal2_r = 21.7

## Table

In [24]:
candidates = pandas.DataFrame({'ra': [gal1_coord.ra.value, 
                                     gal2_coord.ra.value],
                              'dec': [gal1_coord.dec.value, 
                                     gal2_coord.dec.value],
                              'half_light': [0.5, 0.5], # Made up
                               'r_mag': [gal1_r, gal2_r]
                              })
candidates

,ra,dec,half_light,r_mag
0,240.517958,-11.288472,0.5,21.0
1,240.518159,-11.288669,0.5,21.7


## Init in PATH

In [27]:
Path.init_candidates(candidates.ra.values,
                     candidates.dec.values,
                     candidates.half_light.values,
                     mag=candidates.r_mag.values)

# Priors

## Candidate prior

In [29]:
Path.init_cand_prior('inverse', P_U=0.3)  # Trying non-zero P(U) first

## Offset prior

In [30]:
Path.init_theta_prior('exp', 6.)

## Calculate

In [31]:
Path.calc_priors()

array([0.46159993, 0.23840007])

In [32]:
Path.candidates

,ra,dec,ang_size,mag,P_O
0,240.517958,-11.288472,0.5,21.0,0.4616
1,240.518159,-11.288669,0.5,21.7,0.2384


# Posteriors

In [34]:
P_Ox, P_Ux = Path.calc_posteriors('fixed', box_hwidth=30., max_radius=10.)

In [35]:
Path.candidates

,ra,dec,ang_size,mag,P_O,p_xO,P_Ox
0,240.517958,-11.288472,0.5,21.0,0.4616,0.045227,0.900940
1,240.518159,-11.288669,0.5,21.7,0.2384,0.006483,0.066694
